# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
!pip install citipy

In [121]:
# Dependencies
import json
import requests
import pandas as pd
from api_keys import weather_api_key, g_key
from citipy import citipy
import numpy as np
from scipy.stats import linregress
from scipy import stats
from pprint import pprint
import os
import datetime as dt
import time

#file; for the end
data = "../Output/cities.csv"



## Generate Cities List

In [85]:
# Ranges of coords
range_lat = (-90, 90)
range_lng = (-180, 180)

# List for holding coords and cities
coordinates = []
cities = []

#gather random coords
np.random.seed(1000)

lats = np.random.uniform(range_lat[0], range_lat[1], size=2000)
longs = lngs = np.random.uniform(range_lng[0], range_lng[1], size=2000)

coordinates = zip(lats, longs)

#Nearest city
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

# If the city is unique, then add it to cities list
    if city not in cities:
        cities.append(city)
        


In [86]:
# Print the city count to confirm sufficient count
len(cities)

759

In [90]:
# Convert city list to dataframe and rename column to "city"
weather_dict = {
    "City": cities,
}
weather_data = pd.DataFrame(weather_dict)

# Add columns to cities dataframe
weather_data["Cloudiness"] = np.nan
weather_data["Country"] = np.nan
weather_data["Date"] = np.nan
weather_data["Humidity"] = np.nan
weather_data["Lat"] = np.nan
weather_data["Lng"] = np.nan
weather_data["Max Temp"] = np.nan
weather_data["Wind Speed"] = np.nan
weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,dali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bluff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,norman wells,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tabiauea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,leningradskiy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [94]:
# Save config information #reference
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "Imperial"

city = "London"

# Build query URL | F-strings can be used too.
# query_url = f"{url}""
query_url = url + "appid=" + weather_api_key + "&units=" + units + "&q=" +city


# Get weather data
weather_response = requests.get(query_url)
#convert to a json
weather_json = weather_response.json()

# Get the temperature from the response
print(f"The weather API responded with: {weather_json}.")

The weather API responded with: {'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 42.75, 'feels_like': 42.75, 'temp_min': 38.17, 'temp_max': 46.04, 'pressure': 1013, 'humidity': 93}, 'visibility': 10000, 'wind': {'speed': 1.01, 'deg': 246, 'gust': 4}, 'clouds': {'all': 28}, 'dt': 1622003008, 'sys': {'type': 2, 'id': 2019646, 'country': 'GB', 'sunrise': 1622001276, 'sunset': 1622059251}, 'timezone': 3600, 'id': 2643743, 'name': 'London', 'cod': 200}.


In [103]:
weather_json

{'coord': {'lon': 100.3354, 'lat': 5.4112},
 'weather': [{'id': 801,
   'main': 'Clouds',
   'description': 'few clouds',
   'icon': '02n'}],
 'base': 'stations',
 'main': {'temp': 300.57,
  'feels_like': 304.72,
  'temp_min': 299.81,
  'temp_max': 301.42,
  'pressure': 1008,
  'humidity': 87},
 'visibility': 8000,
 'wind': {'speed': 2.68, 'deg': 90, 'gust': 0},
 'clouds': {'all': 20},
 'dt': 1622569055,
 'sys': {'type': 2,
  'id': 2037578,
  'country': 'MY',
  'sunrise': 1622588660,
  'sunset': 1622633355},
 'timezone': 28800,
 'id': 1735106,
 'name': 'George Town',
 'cod': 200}

In [111]:
weather_json['coord']['lon']
weather_json['coord']['lat']
weather_json['clouds']['all']
weather_json['sys']['country']
weather_json['main']['humidity']
weather_json['main']['temp_max']
weather_json['wind']['speed']
# dt.date(weather_json['dt'])

100.3354

5.4112

20

'MY'

87

301.42

2.68

In [110]:
dt.datetime.fromtimestamp(int(weather_json['dt'])).strftime('%Y-%m-%d %H:%M:%S')

'2021-06-01 12:37:35'

In [119]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"

cities_found = []
cloudiness = []
countries = []
dates = []
humidities = []
lats = []
lngs = []
max_temps = []
wind_speeds = []

# Use iterrows to loop through dataframe using the city name
for city in cities:
    time.sleep(1)
    # Build query URL | F-strings can be used too.
    # query_url = f"{url}""
    query_url = url + "appid=" + weather_api_key + "&q=" + city.replace(" ","+")

    # Get weather data
    weather_response = requests.get(query_url)
    #convert to a json
    weather_json = weather_response.json()
    try:
        dates.append(dt.datetime.fromtimestamp(int(weather_json['dt'])).strftime('%Y-%m-%d %H:%M:%S'))
        cloudiness.append(weather_json['clouds']['all'])
        countries.append(weather_json['sys']['country'])
        humidities.append(weather_json['main']['humidity'])
        lats.append(weather_json['coord']['lat'])
        lngs.append(weather_json['coord']['lon'])
        max_temps.append(weather_json['main']['temp_max'])
        wind_speeds.append(weather_json['wind']['speed'])
        cities_found.append(city)
        
        print(f"✅{city} data processing")
    
    except:
        print(f"❌{city} not found")

  
    
    


dali data processing
bluff data processing
norman wells data processing
tabiauea not found
leningradskiy data processing
tuatapere data processing
kaitangata data processing
avarua data processing
vaini data processing
okhotsk data processing
arraial do cabo data processing
chifeng data processing
hithadhoo data processing
tsihombe not found
tasbuget not found
hermanus data processing
sorland data processing
honningsvag data processing
talnakh data processing
georgetown data processing
port alfred data processing
barrow data processing
rikitea data processing
seoul data processing
punta arenas data processing
oktyabrskoye data processing
hasaki data processing
atasu data processing
taburi not found
ilulissat data processing
tromso data processing
ushuaia data processing
mahebourg data processing
aybak not found
zhigansk data processing
saint-philippe data processing
atuona data processing
nikolskoye data processing
san cristobal data processing
busselton data processing
saskylakh data 

ReadTimeout: HTTPConnectionPool(host='api.openweathermap.org', port=80): Read timed out. (read timeout=None)

In [120]:
#sum of countries
countries

97

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [122]:
# cities_found = []
# cloudiness = []
# countries = []
# dates = []
# humidities = []
# lats = []
# lngs = []
# max_temps = []
# wind_speeds = []

# Convert city list to dataframe and rename column to "city"

weather_data = pd.DataFrame({
    "Date": dates,
    "City": cities_found,
    "Country": countries,
    "Cloudiness": cloudiness,
    "Humidity": humidities,
    "Latitude": lats,
    "Longitude": lngs,
    "Maximum Temp": max_temps,
    "Wind Speed": wind_speeds
    
})

weather_data

,Date,City,Country,Cloudiness,Humidity,Latitude,Longitude,Maximum Temp,Wind Speed
0,2021-06-01 13:23:21,dali,CN,16,89,25.7000,100.1833,290.02,2.53
1,2021-06-01 13:23:22,bluff,NZ,99,87,-46.6000,168.3333,283.04,8.07
2,2021-06-01 13:23:22,norman wells,CA,90,81,65.2820,-126.8329,280.17,7.20
3,2021-06-01 13:23:22,leningradskiy,RU,71,89,69.3833,178.4167,270.94,5.90
4,2021-06-01 13:23:23,tuatapere,NZ,82,86,-46.1333,167.6833,283.05,2.52
...,...,...,...,...,...,...,...,...,...
92,2021-06-01 13:20:02,coihaique,CL,75,57,-45.5752,-72.0662,282.12,2.57
93,2021-06-01 13:20:03,dzilam gonzalez,MX,87,45,21.2833,-88.9333,306.53,2.96
94,2021-06-01 13:20:04,bata,GQ,100,91,1.8639,9.7658,297.94,2.33
95,2021-06-01 13:20:04,bengkulu,ID,87,89,-3.8004,102.2655,297.65,1.74


In [124]:
weather_data.to_csv("WeatherPyData.csv")

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [123]:
weather_data[weather_data["Humidity"]>100]

,Date,City,Country,Cloudiness,Humidity,Latitude,Longitude,Maximum Temp,Wind Speed


In [46]:
#  Get the indices of cities that have humidity over 100%.
#no cities found with humidity over 100 - skipping

In [47]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression